In [2]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns

In [25]:
vix_Data = pd.read_csv("resources/vix_data_updated.csv", index_col="dates").drop("target", axis = 1)["VIX"]

In [38]:
vix_Data = pd.DataFrame(vix_Data)

### Train test-> we need to decide what percentage at the end of the data we will be using .
- 20%

In [39]:
test_percent = 0.2

## we now need to calculate the cutoff point - index a 649 will be where our data will be cutoff for testing

In [40]:
test_point = np.round(len(vix_Data) * test_percent)
test_point

649.0

### Set the index for the testing data

In [41]:
test_index = int(len(vix_Data) - test_point)

In [42]:
test_index

2597

### Setting training and test data

In [43]:
train = vix_Data.iloc[:test_index]

In [44]:
test = vix_Data.iloc[test_index:]

### Scaling our data

In [55]:
from sklearn.preprocessing import MinMaxScaler

In [56]:
scalar  = MinMaxScaler()

In [57]:
train_scaled = scalar.fit_transform(train)

In [58]:
test_scaled = scalar.transform(test)

### Lets make batches of data that the RNN will understand

In [59]:
from tensorflow.keras.preprocessing.sequence import TimeseriesGenerator

In [60]:
length = 30
batch = 1
generator = TimeseriesGenerator(train_scaled,train_scaled, length== length, batch_size=batch)

In [61]:
len(train_scaled)

2597

In [62]:
len(generator)

2596